# Advanced Stock Portfolio Analysis using Pandas, NumPy, Matplotlib, Plotly, and yfinance

This notebook provides an advanced toolkit for stock portfolio analysis using real-time data from Yahoo Finance. We include enhanced metrics like Sharpe Ratio, Beta, and portfolio optimization techniques.

## Setup and Imports
Let's start by importing all necessary libraries and setting up our environment.

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import yfinance as yf
from scipy.optimize import minimize

# Matplotlib settings for better visuals
plt.style.use('seaborn-darkgrid')


<ipython-input-1-a3040bd7626f>:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-darkgrid')


## Fetching Data
We'll define a list of stock tickers, including an index for benchmarking, and fetch their historical price data.

In [2]:

# Define stock symbols including an index for benchmarking (S&P 500)
stocks = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', '^GSPC']

# Fetch historical data
data = yf.download(stocks, start='2023-01-01', end='2024-01-01')['Adj Close']
data.head()


[*********************100%%**********************]  5 of 5 completed


Ticker,AAPL,AMZN,GOOGL,MSFT,^GSPC
Date,,,,,
2023-01-03,124.216301,85.820000,89.120003,237.036011,3824.139893
2023-01-04,125.497498,85.139999,88.080002,226.667297,3852.969971
2023-01-05,124.166649,83.120003,86.199997,219.949387,3808.100098
2023-01-06,128.735245,86.080002,87.339996,222.541550,3895.080078
2023-01-09,129.261612,87.360001,88.019997,224.708298,3892.090088


## Risk and Performance Metrics
Calculate the Sharpe Ratio, Beta against S&P 500, and Value at Risk.

In [3]:

# Calculate daily returns
daily_returns = data.pct_change()

# Risk-free rate assumption for Sharpe Ratio (e.g., 1% annual return)
risk_free_rate = 0.01 / 252

# Calculate excess returns
excess_returns = daily_returns.subtract(risk_free_rate, axis=0)

# Sharpe Ratios
sharpe_ratios = excess_returns.mean() / excess_returns.std() * np.sqrt(252)

# Betas (covariance with the market / variance of the market)
market_returns = daily_returns['^GSPC']
betas = daily_returns.cov().loc[:'^GSPC', '^GSPC'] / market_returns.var()

# Value at Risk (95% confidence)
VaR_95 = -daily_returns.quantile(0.05)

print(f"Sharpe Ratios:\n{sharpe_ratios}\n")
print(f"Betas:\n{betas}\n")
print(f"95% VaR:\n{VaR_95}")


Sharpe Ratios:
Ticker
AAPL     2.267186
AMZN     1.886040
GOOGL    1.616569
MSFT     1.938103
^GSPC    1.695601
dtype: float64

Betas:
Ticker
AAPL     1.104513
AMZN     1.538754
GOOGL    1.387064
MSFT     1.179507
^GSPC    1.000000
Name: ^GSPC, dtype: float64

95% VaR:
Ticker
AAPL     0.017177
AMZN     0.027312
GOOGL    0.025086
MSFT     0.022940
^GSPC    0.013771
Name: 0.05, dtype: float64


## Portfolio Optimization
Using Modern Portfolio Theory (MPT) to find the optimal portfolio.

In [6]:
def neg_sharpe(weights, returns, cov_matrix, risk_free_rate):
    portfolio_return = np.dot(weights, returns)
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return -(portfolio_return - risk_free_rate) / portfolio_std_dev

# Daily returns for stocks excluding index
stocks_optimization = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']
returns = daily_returns[stocks_optimization]

# Mean returns and covariance matrix for optimization
mean_returns_optimization = returns.mean()
cov_matrix_optimization = returns.cov()

# Constraints and bounds
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # The sum of weights is 1
bounds = tuple((0, 1) for _ in stocks_optimization)  # Weights between 0 and 1

# Initial guess
initial_guess = np.array(len(stocks_optimization) * [1. / len(stocks_optimization)])

# Risk-free rate assumption (annualized)
annual_risk_free_rate = 0.01
daily_risk_free_rate = annual_risk_free_rate / 252

# Portfolio optimization
opt_results = minimize(neg_sharpe, initial_guess, args=(mean_returns_optimization, cov_matrix_optimization, daily_risk_free_rate),
                       method='SLSQP', bounds=bounds, constraints=cons)

opt_results

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.15817694923715045
       x: [ 6.149e-01  0.000e+00  1.824e-01  2.028e-01]
     nit: 9
     jac: [-3.201e-03 -7.845e-04 -3.216e-03 -3.215e-03]
    nfev: 45
    njev: 9

## Interactive Visualizations
Creating interactive plots using Plotly for a better visual analysis.

In [7]:

# Interactive plot of stock prices
fig_prices = go.Figure()
for stock in stocks:
    fig_prices.add_trace(go.Scatter(x=data.index, y=data[stock], mode='lines', name=stock))
fig_prices.update_layout(title='Stock Prices Over Time', xaxis_title='Date', yaxis_title='Price')
fig_prices.show()

# Interactive plot of cumulative returns
cumulative_returns = (1 + daily_returns).cumprod() - 1
fig_returns = go.Figure()
for stock in stocks:
    fig_returns.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns[stock], mode='lines', name=stock))
fig_returns.update_layout(title='Cumulative Returns Over Time', xaxis_title='Date', yaxis_title='Cumulative Return')
fig_returns.show()
